In [1]:
import os
import sys 
import wget
import shutil
import tensorflow as tf

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = f'http://download.tensorflow.org/models/object_detection/tf2/20200711/{PRETRAINED_MODEL_NAME}.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

# Membuat folder untuk tiap path jika belum ada
for path in paths.values():
    if not os.path.exists(path):
        os.mkdir(path)

# 1. Download Pretrained Models from Tensorflow Model Zoo

In [4]:
os.path.join(paths['APIMODEL_PATH'])

'Tensorflow\\models'

In [5]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}
else:
    print('sudah ada')

sudah ada


In [6]:
# Install Tensorflow Object Detection 
if 'protoc' not in os.listdir('./Tensorflow'):
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

In [7]:
sys.path.append(os.path.abspath("\Tensorflow\models\research\object_detection"))

In [6]:
# VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# # Verify Installation
# !python {VERIFICATION_SCRIPT}

In [8]:
if PRETRAINED_MODEL_NAME not in os.listdir(r'.\Tensorflow\workspace\pre-trained-models'):
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
else:
    print('pretrained model sudah ada')

pretrained model sudah ada


# 2. Create Label Map

In [9]:
COLLECTED_IMG_PATH = r'.\Tensorflow\workspace\images\collectedimages'
labels = []

for id, name in enumerate(os.listdir(COLLECTED_IMG_PATH)):
    labels.append({'name': name, 'id': id+1})

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

In [10]:
mau_update_record = False
if mau_update_record:
    !python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
    !python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 
    print("\033[92m TFRecord telah diupdate :D")
else:
    print('TFRecord tidak diupdate')

TFRecord tidak diupdate


# 4. Pipeline Config Manual for Transfer Learning

In [11]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [12]:
shutil.copy(src=os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config'), 
                dst=os.path.join(paths['CHECKPOINT_PATH']))

config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

### Momentum Optimizer

In [12]:
# Todo: Momentum Optimizer
# Todo: Tuning optimizer and Learning rate for Momentum
# pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = 0.001
# pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = 0.000266

### Adam Optimizer

In [13]:
# # !COMMENT KALAU TIDAK MENGGUNAKAN ADAM MANUAL DECAY
# # Jalankan 1x saja, karena nanti schedulenya bisa nambah terus
# TRAIN_STEPS = 50_000
# SCHEDULED_STEP_BASE = 7500
# SCHEDULED_LR_BASE = 0.002
# NUM_SCHEDULER = int(TRAIN_STEPS // SCHEDULED_STEP_BASE)

# print("Jumlah scheduler : ", NUM_SCHEDULER)

# # Menambah schedule adam optimizer (manual decay)
# for i in range(NUM_SCHEDULER):
#     pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.manual_step_learning_rate.schedule.add()

In [30]:
# Todo: Adam Optimizer
# Todo: Tuning optimizer and Learning rate for Adam (constant learning rate)
# pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.constant_learning_rate.learning_rate = 0.001
# pipeline_config.train_config.optimizer.use_moving_average = False
# pipeline_config.train_config.optimizer.adam_optimizer.epsilon = 1e-08

# Todo: Tuning optimizer and Learning rate for Adam (cosine decay) 
# pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = .001
# pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = TRAIN_STEPS
# pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = .000126666
# pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.cosine_decay_learning_rate.warmup_steps = 1000
# pipeline_config.train_config.optimizer.adam_optimizer.epsilon = 1e-06
# pipeline_config.train_config.optimizer.use_moving_average = False

# Todo: Tuning optimizer and Learning rate for Adam (exponential decay)
pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.exponential_decay_learning_rate.initial_learning_rate = 0.000145
pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.exponential_decay_learning_rate.decay_steps = 9_500
pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.exponential_decay_learning_rate.decay_factor = .95
pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.exponential_decay_learning_rate.min_learning_rate = 1e-012
pipeline_config.train_config.optimizer.adam_optimizer.epsilon = 1e-08
pipeline_config.train_config.optimizer.use_moving_average = False

# Todo: Tuning optimizer and Learning rate for Adam (Manual decay)
# pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.manual_step_learning_rate.initial_learning_rate = 0.003
# TMP_SCHEDULED_STEP_BASE = SCHEDULED_STEP_BASE
# for i in range(NUM_SCHEDULER): 
#     if SCHEDULED_STEP_BASE < TRAIN_STEPS:
#         pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.manual_step_learning_rate.schedule[i].step = SCHEDULED_STEP_BASE
#         pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.manual_step_learning_rate.schedule[i].learning_rate = SCHEDULED_LR_BASE

#         SCHEDULED_STEP_BASE += TMP_SCHEDULED_STEP_BASE
#         SCHEDULED_LR_BASE /= (i+2)

# pipeline_config.train_config.optimizer.adam_optimizer.epsilon = 1e-08
# pipeline_config.train_config.optimizer.use_moving_average = False

### RMSprop

In [15]:
# # Todo: Tuning optimizer and Learning rate for RMSprop (exponential decay)
# pipeline_config.train_config.optimizer.rms_prop_optimizer.learning_rate.exponential_decay_learning_rate.initial_learning_rate = .003
# pipeline_config.train_config.optimizer.rms_prop_optimizer.learning_rate.exponential_decay_learning_rate.decay_steps = 10000
# pipeline_config.train_config.optimizer.rms_prop_optimizer.learning_rate.exponential_decay_learning_rate.decay_factor = .95
# pipeline_config.train_config.optimizer.use_moving_average = False

### Pipeline config

In [31]:
# Menambah augmentasi 
NUM_AUGMENTATION = 3
for i in range(NUM_AUGMENTATION):
    pipeline_config.train_config.data_augmentation_options.add()

In [32]:
TRAIN_STEPS = 75_000
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 6

pipeline_config.model.ssd.post_processing.batch_non_max_suppression.max_detections_per_class =  10
pipeline_config.model.ssd.post_processing.batch_non_max_suppression.max_total_detections = 10
# pipeline_config.train_config.max_number_of_boxes = 10

# pipeline_config.model.ssd.freeze_batchnorm = True

# # Todo: Image augmentation
#? random_adjust_brightness
pipeline_config.train_config.data_augmentation_options[2].random_adjust_brightness.max_delta = 0.2
#? random_adjust_contrast
pipeline_config.train_config.data_augmentation_options[3].random_adjust_contrast.min_delta = 0.8
pipeline_config.train_config.data_augmentation_options[3].random_adjust_contrast.max_delta = 1.1
#? random_adjust_saturation
pipeline_config.train_config.data_augmentation_options[4].random_adjust_saturation.min_delta = 0.8
pipeline_config.train_config.data_augmentation_options[4].random_adjust_saturation.max_delta = 1.1

# Todo: Configure the path
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [33]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

print(config_text)

model {
  ssd {
    num_classes: 8
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.01
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.997
          scale: true
          epsilon: 0.001
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      arg

# 5. Training

In [28]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
train_cmd = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}". \
                format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], TRAIN_STEPS)
print(train_cmd)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\my_ssd_mobnet --pipeline_config_path=Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config --num_train_steps=75000


# 6. Evaluating

In [20]:
eval_cmd = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
print(eval_cmd)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\my_ssd_mobnet --pipeline_config_path=Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config --checkpoint_dir=Tensorflow\workspace\models\my_ssd_mobnet


# 7 Visualize using Tensorboard

In [ ]:
## Train
#         cd .\Tensorflow\workspace\models\my_ssd_mobnet_3\train 
#         tensorboard --logdir=. 

# ## Test
#         cd .\Tensorflow\workspace\models\my_ssd_mobnet_3\eval 
#         tensorboard --logdir=. 

# 8. Load Best Trained Model

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-1')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9.1 Detect from an Image

In [ ]:
# import cv2 
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

In [ ]:
# category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])
# # IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', '20220514_181950.jpg')
# IMAGE_PATH = r'C:\Users\User\Documents\VisualStudioCode\Rebage-Machine-Learning\Development\f1.jpeg'

In [ ]:
# img = cv2.imread(IMAGE_PATH)
# image_np = np.array(img)
# input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

# detections = detect_fn(input_tensor)

# num_detections = int(detections.pop('num_detections'))
# detections = {key: value[0, :num_detections].numpy()
#                 for key, value in detections.items()}
# detections['num_detections'] = num_detections

# # detection_classes should be ints.
# detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# label_id_offset = 1
# image_np_with_detections = image_np.copy()

# val = viz_utils.visualize_boxes_and_labels_on_image_array(
#             image_np_with_detections,
#             detections['detection_boxes'],
#             detections['detection_classes']+label_id_offset,
#             detections['detection_scores'],
#             category_index,
#             use_normalized_coordinates=True,
#             max_boxes_to_draw=5,
#             min_score_thresh=.5,
#             agnostic_mode=False)

# plt.figure(figsize=(14, 10))
# plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)) #comment/uncomment aja
# plt.show() #comment/uncomment aja

In [ ]:
# image_np.shape

In [ ]:
# error terus maka numpy nya di save lalu dijalanin di ipynb 5
# plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
# plt.show()
# np.save('gambar1', val, )

# 9.2 Real Time Detections from Webcam

In [ ]:
import cv2
import numpy as np

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                    for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=10,
                min_score_thresh=.3,
                agnostic_mode=False)

    # print(detections['num_detections'])
    # print(detections['detection_classes'])
    # print()

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

# 10. Freezing the Graph

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')
freeze_graph_cmd = "python {} --input_type=encoded_image_string_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])
print(freeze_graph_cmd)

In [ ]:
!{freeze_graph_cmd}